In [ ]:
!pip install mlflow

In [ ]:
!pip install ipywidgets --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org


In [ ]:
!pip install jupyter_contrib_nbextensions --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

In [11]:
!pip install jupyter contrib nbextension install --user --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

  Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
  Using cached contrib-0.3.0-py3-none-any.whl (16 kB)


ERROR: Could not find a version that satisfies the requirement nbextension (from versions: none)
ERROR: No matching distribution found for nbextension

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install jupyter nbextension enable --py widgetsnbextension --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --py (--pypi-url, --python, --python-version?)


In [ ]:
import mlflow

# mlflow.set_tracking_uri("http://ec2-44-248-43-159.us-west-2.compute.amazonaws.com:8000/")
mlflow.set_tracking_uri("http://ec2-52-12-111-1.us-west-2.compute.amazonaws.com:8000/")

with mlflow.start_run():
    mlflow.log_param("alpha", 0.5)
    mlflow.log_metric("rmse", 0.89)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [4]:
df.isnull().sum()

clean_comment    100
category           0
dtype: int64

In [7]:
df[df['clean_comment'].isnull()]

,clean_comment,category


In [8]:
df.dropna(inplace=True)

In [9]:
df[df['clean_comment'].isnull()]

,clean_comment,category


In [10]:
duplicates=df[df.duplicated()]

In [11]:
duplicates

,clean_comment,category
375,,0
392,,0
617,aurum mom,0
651,,0
1222,,0
...,...,...
36915,who won,0
37044,,0
37125,hari,0
37158,top kek,1


In [12]:
df['clean_comment']=df['clean_comment'].str.strip()

In [13]:
df.drop_duplicates(inplace=True)

In [14]:
df[df.duplicated()]

,clean_comment,category


In [15]:
df[df['clean_comment']== '']

,clean_comment,category
181,,0


In [16]:
df

,clean_comment,category
0,family mormon have never tried explain them th...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [31]:
df.columns

Index(['clean_comment', 'category'], dtype='object')

In [17]:
df.drop(df[df['clean_comment'] == ''].index, inplace=True)

In [18]:
df[df['clean_comment']== '']

,clean_comment,category


In [34]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [35]:
# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Damini.gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Damini.gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [37]:
df['clean_comment'] =df['clean_comment'].apply(preprocess_comment)

In [40]:
df['clean_comment']

0        family mormon never tried explain still stare ...
1        buddhism much lot compatible christianity espe...
2        seriously say thing first get complex explain ...
3        learned want teach different focus goal not wr...
4        benefit may want read living buddha living chr...
                               ...                        
37244                                                jesus
37245    kya bhai pure saal chutiya banaya modi aur jab...
37246                 downvote karna tha par upvote hogaya
37247                                            haha nice
37248                            facebook working bjp cell
Name: clean_comment, Length: 36607, dtype: object

In [25]:
df['category']

0        1
1        1
2       -1
3        0
4        1
        ..
37244    0
37245    1
37246    0
37247    1
37248    0
Name: category, Length: 36607, dtype: int64

In [41]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
vectorizer = CountVectorizer(max_features=10000)

In [43]:
X = vectorizer.fit_transform(df['clean_comment']).toarray()
  # Assuming 'sentiment' is the target variable (0 or 1 for binary classification)

In [44]:
y = df['category']

In [ ]:
X.shape

In [ ]:
y.shape

In [47]:
import mlflow

mlflow.set_tracking_uri("http://ec2-52-12-111-1.us-west-2.compute.amazonaws.com:8000/")

In [48]:
mlflow.set_experiment("RF Baseline")

<Experiment: artifact_location='s3://mlflow-bucket-damini/195026867086354448', creation_time=1758172830170, experiment_id='195026867086354448', last_update_time=1758172830170, lifecycle_stage='active', name='RF Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
with mlflow.start_run() as run:
    # Log a description for the run
    mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
    mlflow.set_tag("experiment_type", "baseline")
    mlflow.set_tag("model_type", "RandomForestClassifier")

    # Add a description
    mlflow.set_tag("description", "Baseline RandomForest model for sentiment analysis using Bag of Words (BoW) with a simple train-test split")

    # Log parameters for the vectorizer
    mlflow.log_param("vectorizer_type", "CountVectorizer")
    mlflow.log_param("vectorizer_max_features", vectorizer.max_features)

    # Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Log metrics for each class and accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):  # For precision, recall, f1-score, etc.
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Confusion matrix plot
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    # Save and log the confusion matrix plot
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")

    # Log the Random Forest model
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Optionally log the dataset itself (if it's small enough)
    df.to_csv("dataset.csv", index=False)
    mlflow.log_artifact("dataset.csv")

# Display final accuracy
print(f"Accuracy: {accuracy}")

In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       1.00      0.01      0.02      1647
           0       0.68      0.80      0.73      2530
           1       0.62      0.85      0.71      3145

    accuracy                           0.64      7322
   macro avg       0.76      0.55      0.49      7322
weighted avg       0.72      0.64      0.56      7322



In [55]:
!pip install --upgrade boto3 --trusted-host pypi.org --trusted-host files.pythonhosted.org

                                              0.0/139.3 kB ? eta -:--:--
     --                                       10.2/139.3 kB ? eta -:--:--
     -----                                 20.5/139.3 kB 217.9 kB/s eta 0:00:01
     -------------                         51.2/139.3 kB 327.7 kB/s eta 0:00:01
     ---------------------                 81.9/139.3 kB 508.4 kB/s eta 0:00:01
     ---------------------                 81.9/139.3 kB 508.4 kB/s eta 0:00:01
     ------------------------------------ 139.3/139.3 kB 549.4 kB/s eta 0:00:00
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.77
    Uninstalling boto3-1.28.77:
      Successfully uninstalled boto3-1.28.77



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
!pip install awscli --trusted-host pypi.org --trusted-host files.pythonhosted.org


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!aws configure